# TABLE DES MATIERES
* [INFORMATIONS ATTENDUES](#INFORMATIONS-ATTENDUES)
* [SOURCES](#SOURCES)
    * [ANNUAIRE](#ANNUAIRE)
    * [BDD TOPO](#BDD-TOPO)
    * [POTENTIEL GISEMENT SOLAIRE BRUT AU BATI](#POTENTIEL-GISEMENT-SOLAIRE-BRUT-AU-BATI)
    * [POTENTIEL SOLAIRE](#POTENTIEL-SOLAIRE)
* [EVALUATION](#EVALUATION)
    * [COMPLETUDE](#COMPLETUDE)
    * [PROXIMITE](#PROXIMITE)
        * [Similarité cosinus entre deux textes](#Similarité-cosinus-entre-deux-textes) 

Références: 
- https://outline.services.dataforgood.fr/doc/contraintes-a-respecter-pour-linstallation-de-panneaux-solaires-W2W2IaNwKn
- https://data-iau-idf.opendata.arcgis.com/datasets/iau-idf::le-potentiel-solaire-des-toitures/about
- https://outline.services.dataforgood.fr/doc/ressources-algorithme-9afGJyytC8
- https://geocatalogue.apur.org/catalogue/srv/fre/catalog.search#/metadata/urn:apur:potentiel_gisement_solaire_brut_au_bati

# INFORMATIONS ATTENDUES

## Contraintes à respecter (Nocodb)
- Logistique
    - Pente du toit
    - Orientation au soleil
    - Latitude
    - Indice de rayonnement solaire local
    - Bâtiments classés
    - Bâtiments en zone non classée
    - Ombre (Bâtiments adjacents & végétations)
    - Eléments présents sur le toit
    - Structure porteuse (est-ce que le bâtiment peut supporter le poids?)
- Réseaux

# Potentiel Solaire - Attributs importants
- surf_util : Surface présentant une irradiation suffisante (> 900 kWh/m².an) et sans obstacles (comme des cheminées, velux, aérations, etc.). Cette surface est réellement utilisable pour l'installation de panneaux solaires.
- indic2 : Valeur numérique correspondant au gisement solaire. Les valeurs possibles sont :
1 : Non favorable
1 : Faible
2 : Intermédiaire
3 : Important
- gisement : Classification du gisement solaire en termes qualitatifs : "important", "intermédiaire", "faible", ou "Non favorable".
- moyenne2 : Potentiel solaire moyen au m² du bâtiment, exprimé en kWh/m².an. Cet indicateur représente la quantité d'énergie solaire reçue sur la surface utile.
- production : La production annuelle théorique en kWh/an, qui est une estimation de la quantité d'énergie solaire que la toiture pourrait produire sur une année.
- forme : Forme de la toiture estimée à partir d'un modèle numérique de surface. La forme de la toiture peut influencer l'ensoleillement et donc le potentiel solaire.
- eq_pano : La surface utile en équivalent panneaux solaires. Cet attribut indique le nombre de panneaux solaires susceptibles d'être installés sur la toiture.
- eq_surf : L'équivalent en superficie de l'équivalent panneau solaire, qui donne une idée de la surface disponible pour l'installation de panneaux solaires.

# Qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO
"un des points importants à regarder en l'état c'est la qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO. Pour le moment c'est une règle sur la proximité géographique qui est utilisée mais il y a d'autres métadonnées qui peuvent nous éclairer je pense. Par exemple :
le champs toponyme du layer zone d'activité de la BD TOPO (ex : "Ecole Elementaire Victor Hugo") avec le champs nom_etablissement de l'annuaire (ex: "Ecole élémentaire Victor Hugo")
le champs nature du layer zone d'activité de la BD TOPO (9 valeurs possibles) avec type_etablissement de l'annuaire (8 valeurs possibles)
En creusant, c'est clairement pas l'égalité à chercher mais peut etre calculer la proximité des valeurs (ou juste les comparer) pour voir si on s'est pas complètement loupé. Ou trouver une piste pour les établissements de l'annuaire sans surface d'activité.
Sachant que j'ai pas regardé tous les champs et il y a peut être d'autres informations à utiliser."

# SOURCES

## ANNUAIRE

In [ ]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona
import pandas as pd

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

In [ ]:
!extract-sample-data

In [ ]:
annuaire = gpd.read_file(DATA_FOLDER / 'fr-en-annuaire-education.geojson')
annuaire = annuaire.to_crs(4326)
#annuaire.shape
annuaire[annuaire.code_commune=='93066'].sample(15) # Saint-Denis

## BDD TOPO

In [ ]:
GPKG = DATA_FOLDER / "BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg"
communes = gpd.read_file(GPKG, layer="commune")

# on filtre sur saint-denis
saint_denis = communes[communes.code_insee == "93066"].to_crs(4326)


In [ ]:
# Exploration des layers
layers = fiona.listlayers(GPKG)
layers

In [ ]:
zone_d_activite=gpd.read_file(GPKG, layer="zone_d_activite_ou_d_interet")

In [ ]:
zone_d_activite[zone_d_activite.categorie=='Science et enseignement']

## POTENTIEL GISEMENT SOLAIRE BRUT AU BATI


Les attributs importants pour le projet sont en **gras** 

- N_SQ_EB: Identifiant informatique séquentiel
- **C_CAINSEE: Code INSEE de la commune/arrondissement**
- **C_ENS_MOY: Ensoleillement annuel moyen (KWh/m²/an)**
- M2_E_IN700: Surface d'ensoleillement inférieure à 700 KWh/m²/an
- M2_E_70_80: Surface d'ensoleillement comprise entre 700 et 800 KWh/m²/an
- M2_E_80_90: Surface d'ensoleillement comprise entre 800 et 900 KWh/m²/an
- **M2_E_90_10: Surface d'ensoleillement comprise entre 900 et 1000 KWh/m²/an**
- **M2_E_S1000: Surface d'ensoleillement supérieure à 1000 KWh/m²/an**
- **M2_E_TOT: Surface totale d'ensoleillement (m²)**
- SHAPE_LENGTH: Périmètre de l’emprise (m)
- SHAPE_AREA: Surface de l’emprise (m²)

In [ ]:
potentiel_gisement_solaire = gpd.read_file(DATA_FOLDER / 'potentiel-gisement-solaire-brut-au-bati.geojson')
potentiel_gisement_solaire = potentiel_gisement_solaire.to_crs(4326)

In [ ]:
potentiel_gisement_solaire[potentiel_gisement_solaire.c_cainsee==93066]

## POTENTIEL SOLAIRE

In [ ]:
potentiel = gpd.read_file(DATA_FOLDER / 'potentiel-solaire.geojson')

In [ ]:
potentiel

# EVALUATION

## COMPLETUDE

### Valeurs manquantes

In [ ]:
taux_noms_etablissements_manquants=annuaire.nom_etablissement.isnull().sum()/len(annuaire.nom_etablissement)
print(f"Taux de noms d'établissement manquants: {taux_noms_etablissements_manquants} %")

In [ ]:
taux_toponymes_manquants=zone_d_activite.toponyme.isnull().sum()/len(zone_d_activite.toponyme)
print(f"Taux de toponymes*100 manquants: {taux_toponymes_manquants*100} %") 

### Couverture temporelle

In [ ]:
annuaire[(annuaire.type_etablissement=='Ecole') & (annuaire.code_commune=='93066')].date_maj_ligne.unique()

In [ ]:
zone_d_activite[zone_d_activite.categorie=='Science et enseignement'][["date_modification"]].sort_values(by='date_modification',ascending=False)

## PROXIMITE

Pour voir si les informations sont bien correlées, essayons de calculer la proximité des valeurs (ou juste les comparer) entre sources de données 

In [ ]:
# recherche avec suppression préalable des accents (normalisation + encodage + desencodage + minuscule) 
s_annuaire_ecole_1=annuaire[annuaire.nom_etablissement.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()=="ecole elementaire daniel sorano"]
s_annuaire_ecole_1

In [ ]:
s_bddtopo_ecole_1=zone_d_activite[zone_d_activite.toponyme.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.contains("ecole elementaire daniel sorano")==True]
s_bddtopo_ecole_1

In [ ]:
s_annuaire_ecole_1[['nom_etablissement','type_etablissement']]

In [ ]:
s_bddtopo_ecole_1[['toponyme','nature']]

### Similarité cosinus entre deux textes

Nous calculons la similarité cosinus entre deux attributs texte pour déterminer le taux de similarité  
Plus la valeur de la similarité cosinus est :  
- proche de 1, plus les textes sont similaires.  
- proche de 0, plus les textes sont différents.

In [ ]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from unidecode import unidecode

In [ ]:
def similarite_cosinus(X,Y):
    # Programme pour mesurer la similarité entre
    # deux phrases en utilisant la similarité cosinus (similarité=1 -> identique, similarité=0 -> pas du tout similaire)

    # Suppression des accents et passage en minuscules
    X = unidecode(X).lower()
    Y = unidecode(Y).lower()
    
    # Tokenisation
    X_list = word_tokenize(X)
    Y_list = word_tokenize(Y)
    
    # sw contient la liste des mots vides
    sw = set(stopwords.words('french'))
    l1 = []
    l2 = []
    
    # Supprimer les mots vides de la chaîne
    X_set = {w for w in X_list if w not in sw}
    Y_set = {w for w in Y_list if w not in sw}
    
    # Former un ensemble contenant les mots-clés des deux chaînes
    rvector = X_set.union(Y_set)
    for w in rvector:
        if w in X_set:
            l1.append(1)  # créer un premier vecteur
        else:
            l1.append(0)
        if w in Y_set:
            l2.append(1) # créer un deuxieme vecteur
        else:
            l2.append(0)
    c = 0
    
    # Formule du cosinus
    for i in range(len(rvector)):
        c += l1[i] * l2[i]
    cosine = c / float((sum(l1) * sum(l2)) ** 0.5)
    return cosine

In [ ]:
X=s_annuaire_ecole_1['nom_etablissement'].values[0]
Y=s_bddtopo_ecole_1['toponyme'].values[0]

print(f"similarité ({X}|{Y}): {similarite_cosinus(X,Y)}")

Les attributs annuaire.nom_etablissement et bddtop.toponyme sont identiques

In [ ]:
X=s_annuaire_ecole_1['type_etablissement'].values[0]
Y=s_bddtopo_ecole_1['nature'].values[0]

print(f"similarité ({X}|{Y}): {similarite_cosinus(X,Y)}")